In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy.sparse import save_npz, csr_matrix
from implicit.als import AlternatingLeastSquares
import pickle

C:\Users\79173\Desktop\lct\LCT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Загружаем исходный датасет с посещаемостью занятий

In [2]:
attend_df = pd.read_csv('datasets/attend.csv')
attend_df.head()

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00


### Создаем обучающую выборку

In [3]:
train_df = attend_df
train_df

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
...,...,...,...,...,...,...,...,...,...
5901269,402103132,801371145,101421020,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901270,402103132,801371145,101359314,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901271,402103132,801371145,101357904,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901272,402103132,801371145,101383123,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00


### Создаем список групп, занимающихся онлайн

In [4]:
online_groups_list = train_df[train_df['онлайн/офлайн'] == 'Да']['уникальный номер группы'].unique()
online_groups_list

array([801346550, 801346551, 801346554, ..., 801370220, 801373302,
       801373866], dtype=int64)

### Дополнительная обработка датасета для удобства работы

In [5]:
train_df = train_df.drop(['уникальный номер занятия', 'направление 2', 'направление 3', 'онлайн/офлайн', 'дата занятия', 'время начала занятия', 'время окончания занятия'], axis=1)
train_df = train_df.rename(columns={'уникальный номер группы' : 'item_id', 'уникальный номер участника' : 'user_id'})
train_df = train_df.iloc[:,[1,0]]
train_df.head()

,user_id,item_id
0,101352023,801346550
1,101385462,801346550
2,101421897,801346550
3,101354499,801346550
4,101421312,801346550


### Добавление колонки "attend", отражающей кол-во посещений определенным пользователем определенного мероприятия

In [6]:
train_df = train_df.groupby(['user_id', 'item_id']).size().reset_index().rename(columns={0:'attends'})
train_df.head()

,user_id,item_id,attends
0,101346549,801357282,1
1,101346549,801361690,1
2,101346549,801365191,2
3,101346549,801366199,1
4,101346549,801367532,2


### Удаление из обучающего набора записей с кол-вом посещений < 3

In [7]:
train_df['attends'] = train_df['attends'].map(lambda x: 0 if x < 3 else x)
train_df = train_df[train_df['attends'] > 0]
train_df

,user_id,item_id,attends
11,101346552,801346743,45
12,101346552,801347494,39
13,101346552,801348042,46
14,101346552,801349709,68
16,101346552,801350310,55
...,...,...,...
600355,101449471,801354088,3
600360,101449473,801354088,3
600380,101449494,801367755,3
600383,101449496,801368116,3


### Увеличение количества посещения групп оффлайн, для большего их продвижения в топе

In [8]:
train_df.loc[train_df.item_id.isin(online_groups_list) == False, 'attends'] = train_df.loc[train_df.item_id.isin(online_groups_list) == False, 'attends'] * 2
train_df

,user_id,item_id,attends
11,101346552,801346743,45
12,101346552,801347494,39
13,101346552,801348042,46
14,101346552,801349709,68
16,101346552,801350310,55
...,...,...,...
600355,101449471,801354088,6
600360,101449473,801354088,6
600380,101449494,801367755,3
600383,101449496,801368116,6


### Создание словарей которые помогут "расшифровывать" ответы модели

In [9]:
unique_users = train_df.user_id.unique()
unique_items = train_df.item_id.unique()
item_to_id = {j: i for i, j in enumerate(unique_items)}
id_to_item = {j: i for i, j in item_to_id.items()}
user_to_id = {j: i for i, j in enumerate(unique_users)}
id_to_user = {j: i for i, j in user_to_id.items()}
print('Индекс создан: %d строк %d столбцов' % (len(user_to_id), len(item_to_id)))

Индекс создан: 50785 строк 23444 столбцов


### Создание разреженной матрицы размером <Кол-во пользователей х Кол-во групп> для обучения на ней модели

In [151]:
DT = datetime.datetime.now().strftime('%Y-%m-%d')

num_rows = len(user_to_id)
num_cols = len(item_to_id)
entries = np.ones(train_df.shape[0])
rows = tuple(user_to_id[i] for i in train_df.user_id.values)
cols = tuple(item_to_id[i] for i in train_df.item_id.values)

train_set_csr = csr_matrix(
    (entries, (rows, cols)),
    shape=(num_rows, num_cols),
    dtype=np.float32
)

save_npz(f'model_req/train_set_{DT}.npz', train_set_csr)
print('Данные сохранены в %s' % f'model_req/train_set_{DT}_new_upd.npz')
train_set_csr

Данные сохранены в model_req/train_set_2023-05-27_new_upd.npz


<50785x23444 sparse matrix of type '<class 'numpy.float32'>'
	with 401071 stored elements in Compressed Sparse Row format>

### Обучение модели

In [265]:
implict_als_params = {'factors': 2048, 'iterations': 100}
model = AlternatingLeastSquares(**implict_als_params)

model.fit(train_set_csr.tocsr())

100%|█████████████████████████████████████████████████████████████████| 100/100 [38:39<00:00, 23.20s/it, loss=0.000153]


### Пример работы модели

In [204]:
random_history = train_set_csr[
    np.random.randint(low=0, high=train_set_csr.shape[0])
]

recommends = model.recommend(
    userid = 0,
    user_items=random_history,
    N=10,
    filter_already_liked_items=True,
    recalculate_user=True
)

recommends

(array([  10, 3340, 8166, 2156, 3860, 2449, 1480, 7900, 9511, 8343]),
 array([0.43088698, 0.0613239 , 0.04930925, 0.04537989, 0.04533011,
        0.04461147, 0.04347392, 0.04284901, 0.04117545, 0.04085266],
       dtype=float32))

### Загрузка подготовленного датасета с информацией о группах

In [154]:
groups_df = pd.read_csv('datasets/groups.csv')
groups_df.head()

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,округ площадки,район площадки,расписание в активных периодах,расписание в закрытых периодах,расписание в плановом периоде
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN
1,801356857,Физическая активность,ОФП,ОФП,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 10:00-11:...",NaN
2,801351684,Физическая активность,ОФП,ОФП,"г. Москва, Базовская улица, дом 15, строение 1...","Северный административный округ, Северный адми...","муниципальный округ Западное Дегунино, муницип...",NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 19:00-20:...",NaN
3,801353683,Физическая активность,ОФП,ОФП,"город Москва, улица Обручева, дом 28А, город М...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 13:30-14:...",NaN
4,801352164,Физическая активность,ОФП,ОФП,"город Москва, Воронцовский парк, дом 3, город ...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 10.01.2023 по 28.02.2023, Вт., Пт. 12:00-13:...",NaN


### Функция, возвращающая  данные о группых, рекомендованных конкретному пользователю

In [399]:
def model_recommend(user_ind: int, rec_num: int):
    recommends = model.recommend(
        userid = user_to_id[user_ind],
        user_items=train_set_csr[user_to_id[user_ind]],
        N = rec_num,
        filter_already_liked_items=True,
        recalculate_user=True
    )[0]
    
    rec_list = [id_to_item[x] for x in recommends]
    
    return groups_df[groups_df['уникальный номер'].isin(rec_list)]

In [400]:
model_recommend(101346581, 10)

,уникальный номер,направление 1,направление 2,направление 3,адрес площадки,округ площадки,район площадки,расписание в активных периодах,расписание в закрытых периодах,расписание в плановом периоде
4463,801353625,Физическая активность,Гимнастика,Суставная гимнастика,"город Москва, город Зеленоград, площадь Колумб...",Зеленоградский административный округ,муниципальный округ Старое Крюково,NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 11:30-12:...",NaN
7873,801356686,Физическая активность,Гимнастика,Цигун,"город Москва, город Зеленоград, корпус 1651",Зеленоградский административный округ,муниципальный округ Крюково,NaN,"c 04.04.2022 по 27.07.2022, Пн., Ср. 12:00-13:...",NaN
7876,801347326,Танцы,Танцы,Восточные танцы,"город Москва, город Зеленоград, корпус 1651",Зеленоградский административный округ,муниципальный округ Крюково,NaN,"c 04.04.2022 по 31.12.2022, Пн., Ср. 11:00-12:...",NaN
7887,801356563,Физическая активность,"Фитнес, тренажеры",Пилатес,"город Москва, город Зеленоград, корпус 1006Б, ...","Зеленоградский административный округ, Зеленог...","муниципальный округ Силино, муниципальный окру...",NaN,"c 04.04.2022 по 01.10.2022, Ср. 14:00-15:00, б...",NaN
12440,801351972,Физическая активность,Гимнастика,Цигун,"город Москва, город Зеленоград, корпус 1651, г...","Зеленоградский административный округ, Зеленог...","муниципальный округ Крюково, муниципальный окр...",NaN,"c 06.09.2022 по 31.12.2022, Пн., Ср. 12:00-13:...",NaN
14079,801359714,Физическая активность,"Фитнес, тренажеры",Пилатес,"город Москва, город Зеленоград, корпус 928",Зеленоградский административный округ,муниципальный округ Старое Крюково,NaN,"c 03.10.2022 по 31.12.2022, Пн., Ср. 14:00-15:...",NaN
14082,801359707,Танцы,Танцы,Бальные танцы,"город Москва, город Зеленоград, корпус 928",Зеленоградский административный округ,муниципальный округ Старое Крюково,NaN,"c 03.10.2022 по 31.12.2022, Пн., Ср. 13:00-14:...",NaN
17548,801363022,Физическая активность,Гимнастика,Цигун,"город Москва, город Зеленоград, корпус 1651",Зеленоградский административный округ,муниципальный округ Крюково,"c 09.01.2023 по 27.12.2023, Пн., Ср. 12:00-13:...",NaN,NaN
17556,801367444,Танцы,Танцы,Восточные танцы,"город Москва, город Зеленоград, корпус 1651",Зеленоградский административный округ,муниципальный округ Крюково,"c 09.01.2023 по 27.12.2023, Пн., Ср. 11:00-12:...",NaN,NaN
17561,801363248,Танцы,Танцы,Бальные танцы,"город Москва, город Зеленоград, корпус 928",Зеленоградский административный округ,муниципальный округ Старое Крюково,"c 09.01.2023 по 27.12.2023, Пн., Ср. 13:00-14:...",NaN,NaN


### Сравнение предсказаний модели с реальными данными

In [384]:
attend_df[attend_df['уникальный номер участника'] == 101346581]

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
790610,401432800,801357485,101346581,Танцы,Бальные танцы,Нет,2022-04-14,14:00:00,15:00:00
790800,401432829,801357485,101346581,Танцы,Бальные танцы,Нет,2022-04-21,14:00:00,15:00:00
793604,401433118,801357485,101346581,Танцы,Бальные танцы,Нет,2022-04-28,14:00:00,15:00:00
794240,401433215,801357509,101346581,Танцы,Восточные танцы,Нет,2022-04-14,13:00:00,14:00:00
794339,401433236,801357509,101346581,Танцы,Восточные танцы,Нет,2022-04-21,13:00:00,14:00:00
...,...,...,...,...,...,...,...,...,...
5363266,402046794,801356495,101346581,Танцы,Восточные танцы,Нет,2023-01-12,14:00:00,15:00:00
5365476,402047059,801367180,101346581,Танцы,Восточные танцы,Нет,2023-01-12,12:00:00,13:00:00
5402859,402051359,801356495,101346581,Танцы,Восточные танцы,Нет,2023-01-16,14:00:00,15:00:00
5433055,402054672,801367180,101346581,Танцы,Восточные танцы,Нет,2023-01-17,12:00:00,13:00:00


In [278]:
train_df['user_id'].unique()[10:50]

array([101346576, 101346579, 101346581, 101346582, 101346585, 101346593,
       101346594, 101346596, 101346597, 101346601, 101346603, 101346604,
       101346605, 101346610, 101346611, 101346612, 101346613, 101346615,
       101346617, 101346621, 101346622, 101346623, 101346626, 101346628,
       101346631, 101346632, 101346633, 101346635, 101346638, 101346639,
       101346641, 101346644, 101346645, 101346647, 101346648, 101346649,
       101346650, 101346651, 101346654, 101346657], dtype=int64)

### Сохраняем модель и словари, которые могугт понадобиться в будущем

In [348]:
f = open(f"model_req/actual_model.pkl", "wb")
pickle.dump(model, f)
f.close()

In [14]:
with open('model_req/user_to_id.pickle', 'wb') as handle:
    pickle.dump(user_to_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_req/id_to_item.pickle', 'wb') as handle:
    pickle.dump(id_to_item, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('model_req/id_to_user.pickle', 'wb') as handle:
    pickle.dump(id_to_user, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('model_req/item_to_id.pickle', 'wb') as handle:
    pickle.dump(item_to_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
train_df[train_df.item_id.isin([801346550, 801346551, 801346566, 801362782, 801366892, 801373705])]

,user_id,item_id,attends
4971,101347275,801362782,3
4979,101347275,801366892,4
12605,101348358,801346551,9
12855,101348394,801346551,10
13926,101348565,801362782,5
...,...,...,...
584043,101441388,801366892,5
587713,101442747,801346566,4
591043,101443981,801373705,3
591489,101444142,801346550,4
